In [ ]:
%load_ext autoreload

%autoreload 1

In [24]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from copy import deepcopy
import sys

sys.path.append('../protosp03/recommendation/')
sys.path.append('../protosp03/data/synthetic/')
%aimport matchings
%aimport market
%aimport upskillings
%aimport make_synthetic

In [48]:
path = "../data/taxonomy/taxonomy_V4.csv"
mastery_levels = [1, 2, 3, 4]
years = [i for i in range(2023, 2017, -1)]

In [50]:
skills, learners, jobs = make_synthetic.get_job_market(path, mastery_levels, years )

# Skill Supply

In [ ]:
def get_skill_supply(learners, years):
    skill_supply = {year: Counter() for year in years}
    for learner in learners:
        for skill, level in learner["possessed_skills"].items():
            skill_supply[learner["year"]][(skill, level)] += 1
    return skill_supply


skill_supply = get_skill_supply(learners, years)
skill_supply[2023].most_common(10)

# Skill Demand

In [ ]:
def get_skill_demand(jobs, years):
    skill_demand = {year: Counter() for year in years}
    for job in jobs:
        for skill, level in job["required_skills"].items():
            skill_demand[job["year"]][(skill, level)] += 1
    return skill_demand


skill_demand = get_skill_demand(jobs, years)
skill_demand[2023].most_common(10)

# Skill Trend

In [ ]:
def get_skill_trend(skill_demand, skill, years):
    current_year = years[0]
    last_year = years[1]
    current_demand = skill_demand[current_year][skill]
    last_demand = skill_demand[last_year][skill]
    if last_demand == 0:
        print("last year demand is 0")
        return None
    return 100 * (current_demand - last_demand) / last_demand

# Skill Attractiveness

In [ ]:
def get_skill_attractiveness(skill, years, skill_supply, skill_demand):
    skill_attractiveness = 0
    normalization_factor = 0
    for i, year in enumerate(years):
        if skill in skill_supply[year]:
            skill_attractiveness += skill_demand[year][skill] / (
                skill_supply[year][skill] * (i + 1)
            )
        normalization_factor += 1 / (i + 1)
    return skill_attractiveness / normalization_factor

In [ ]:
def get_learner_attractiveness(learner, years, skill_supply, skill_demand):
    learner_attractiveness = dict()
    for skill, level in learner["possessed_skills"].items():
        learner_attractiveness[skill] = get_skill_attractiveness(
            (skill, level), years, skill_supply, skill_demand
        )
    return learner_attractiveness

# upskilling advice

In [ ]:
matching = matchings.learner_job_matching(learner, job)
new_matching = sys.maxsize
while new_matching > matching:
    upskilling_advice = upskillings.up_skilling_advice(
        learners[0],
        jobs[0],
        market.get_all_skills_attractiveness(
            skills, mastery_levels, years, skill_supply, skill_demand
        ),
    )
    learners[0]["possessed_skills"][upskilling_advice[0]] = upskilling_advice[1]

In [ ]:
jobs[0]

In [ ]:
learner = {
    "possessed_skills": {
        "Business- und Requirements-Engineerin": 1,
        "Mitfühlen": 3,
        "Psychische Fertigkeiten": 3,
    },
    "year": 2023,
}

In [ ]:
skills_attractiveness = market.get_all_skills_attractiveness(
    skills, mastery_levels, years, skill_supply, skill_demand
)

In [ ]:
skills_attractiveness[
    ("Business- und Requirements-Engineerin", 2)
], skills_attractiveness[("Anleiten und Begleiten", 2)]

In [ ]:
skills_attractiveness = dict()
for skill in skills:
    for level in mastery_levels:
        skills_attractiveness[(skill, level)] = get_skill_attractiveness(
            (skill, level), years, skill_supply, skill_demand
        )

In [ ]:
skills_attractiveness = market.get_all_skills_attractiveness(
    skills, mastery_levels, years, skill_supply, skill_demand
)

In [ ]:
upskillings.up_skilling_advice(learners[0], jobs[0], skills_attractiveness)

In [ ]:
learners[0], jobs[0]

# Courses

In [ ]:
def get_random_provided_skills(
    skills, mastery_levels, required_skills, n_provided_skills
):
    provided_skills = dict()
    while len(provided_skills) < n_provided_skills:
        candidate_skill = random.choice(skills)
        candidate_level = random.choice(mastery_levels)
        if (
            candidate_skill not in required_skills
            and candidate_skill not in provided_skills
        ):
            provided_skills[candidate_skill] = candidate_level
        elif (
            candidate_skill in required_skills
            and candidate_level > required_skills[candidate_skill]
        ):
            provided_skills[candidate_skill] = candidate_level

    return provided_skills


def get_random_course(
    skills,
    mastery_levels,
    min_n_required_skills=1,
    max_n_required_skills=5,
    min_n_provided_skills=1,
    max_n_provided_skills=2,
):
    n_required_skills = random.randint(min_n_required_skills, max_n_required_skills)
    required = {
        skill: level
        for skill, level in zip(
            np.random.choice(skills, n_required_skills, replace=False),
            np.random.choice(mastery_levels, n_required_skills, replace=True),
        )
    }

    n_provided_skills = random.randint(min_n_provided_skills, max_n_provided_skills)
    provided = get_random_provided_skills(
        skills, mastery_levels, required, n_provided_skills
    )

    return {"required_skills": required, "provided_skills": provided}


def get_all_courses(
    skills,
    mastery_levels,
    min_n_required_skills=1,
    max_n_required_skills=5,
    min_n_provided_skills=1,
    max_n_provided_skills=2,
    n_courses=1000,
):
    return [
        get_random_course(
            skills,
            mastery_levels,
            min_n_required_skills=1,
            max_n_required_skills=5,
            min_n_provided_skills=1,
            max_n_provided_skills=2,
        )
        for _ in range(n_courses)
    ]


courses = get_all_courses(
    skills,
    mastery_levels,
    min_n_required_skills=1,
    max_n_required_skills=5,
    min_n_provided_skills=1,
    max_n_provided_skills=2,
    n_courses=1000,
)

courses[0]

# Learner-Course Matching

In [ ]:
def learner_course_required_matching(learner, course):
    required_matching = 0
    for skill in course["required_skills"]:
        if skill in learner["possessed_skills"]:
            sim = (
                min(
                    learner["possessed_skills"][skill], course["required_skills"][skill]
                )
                / course["required_skills"][skill]
            )
            required_matching += sim
    return required_matching / len(course["required_skills"])


def learner_course_provided_matching(learner, course):
    provided_matching = 0
    for skill in course["provided_skills"]:
        if skill in learner["possessed_skills"]:
            sim = (
                min(
                    learner["possessed_skills"][skill], course["provided_skills"][skill]
                )
                / course["provided_skills"][skill]
            )
            provided_matching += sim
    return provided_matching / len(course["provided_skills"])


def learner_course_matching(learner, course):
    required_matching = learner_course_required_matching(learner, course)
    provided_matching = learner_course_provided_matching(learner, course)

    if provided_matching >= 1.0:
        return 0

    return required_matching / (provided_matching + 1)

In [ ]:
nb_matching = 0
for learner, course in zip(learners, courses):
    matching = learner_course_matching(learner, course)
    if matching >= 0.1:
        print(f"Matching: {matching:.2f}")
        print(f"Course required: {course['required_skills']}")
        print(f"Course provided: {course['provided_skills']}")
        print(f"Learner: {learner['possessed_skills']}")
        nb_matching += 1
    if nb_matching > 5:
        break

# Learner-skill Achievability

In [ ]:
def learner_skill_achievability(learner, skill, mastery_level, courses):
    achievability = 0
    nb_courses = 0
    for course in courses:
        if (
            skill in course["provided_skills"]
            and mastery_level == course["provided_skills"][skill]
        ):
            achievability += learner_course_required_matching(learner, course)
            nb_courses += 1
    return achievability, nb_courses

In [ ]:
learners[0]

In [ ]:
jobs[0]

In [ ]:
nb_achievable = 0

for learner in learners:
    achievability, nb_courses = learner_skill_achievability(
        learner, "Motivieren", 1, courses
    )
    if achievability > 0:
        nb_achievable += 1
        print(f"Achievability: {achievability:.2f}")
        print(f"Number of courses: {nb_courses}")
        print(f"Learner: {learner['possessed_skills']}")
        print(f"Skill: Motivieren")
        if nb_achievable > 5:
            break

In [ ]:
def max(a):
    max = a[0]
    for i in range(1, len(a)):
        if a[i] > max:
            max = a[i]
    return max


max([7, 8, 3, 4, 5])